In [1]:
#initialize code and import packages
import dask
import os
from dask.distributed import LocalCluster, Client, wait
import os
import convenient as cv
from mapping_functions import *
import xarray as xr
import numpy as np

In [ ]:
##FOR USE ON SCRATCH
scheduler_file = os.path.join(os.environ["SCRATCH"], "scheduler_file.json")

dask.config.config["distributed"]["dashboard"]["link"] = "{JUPYTERHUB_SERVICE_PREFIX}proxy/{host}:{port}/status" 

client = Client(scheduler_file=scheduler_file)
client

In [ ]:
client.upload_file("njit_options.py")
client.upload_file("basisfunctions.py")
client.upload_file("distancefunctions.py")
client.upload_file("mean_field_lsq.py")

In [ ]:
#TESTING ON HOMES ONLY
cluster = LocalCluster()  # Launches a scheduler and workers locally
client = Client(cluster)  # Connect to distributed cluster and override default
print('External dashboard-link: ', client.dashboard_link)
print(client)

Temperature and Salinity Mean Fields

In [2]:
window_size_betaparam = math.sqrt(1.0 * 700e3**2)
NHarm = 2
testn = 0
datafile = '/global/homes/k/kefalc/code/data_files/Final/floats_years_2015_2019_temperature_salinity_oxygen_10daily_SO_30S_edit.nc'
maskfile = '/global/homes/k/kefalc/code/gpr-mapping-mean/SO_mask.nc'

time_i = (np.datetime64('2015-01-01') - np.datetime64('1970-01-01T00:00:00', 'ns')) / np.timedelta64(1, 'ns') / 1e9 / 86400.0
time_f = (np.datetime64('2019-12-31') - np.datetime64('1970-01-01T00:00:00', 'ns')) / np.timedelta64(1, 'ns') / 1e9 / 86400.0

grid_time_i = (np.datetime64('2017-01-01') - np.datetime64('1970-01-01T00:00:00', 'ns')) / np.timedelta64(1, 'ns') / 1e9 / 86400.0
grid_time_f = (np.datetime64('2017-12-31') - np.datetime64('1970-01-01T00:00:00', 'ns')) / np.timedelta64(1, 'ns') / 1e9 / 86400.0

In [4]:
%%time

for plevel in range(0,7): # Select pressurelevels
    #betaparams
    betaparamfile= '/global/homes/k/kefalc/code/anom_files/'+'/betaparam_plevel'+str(plevel)+'_dates'+str(time_i)+'_'+str(time_f)+'_test'+str(testn)+'.nc'
    #ts_betaparam( plevel, datafile, betaparamfile, maskfile, window_size_betaparam, window_size_betaparam, NHarm, client)
    
    #large scale mean/residuals
    residualfile = '/global/homes/k/kefalc/code/anom_files/test'+'/residualfile_plevel'+str(plevel)+'_dates'+str(time_i)+'_'+str(time_f)+'_test'+str(testn)+'.nc'
    #interpolate_to_observations_TS( datafile, betaparamfile, residualfile, plevel )
    
    #gridded mean/residuals
    ts_gridmeanfile = '/global/homes/k/kefalc/code/anom_files/test'+'/grid_TS_mean_ws'+str(np.int64(window_size_betaparam/1e3))+'_plevel'+str(plevel)+'_dates'+str(grid_time_i)+'_'+str(grid_time_f)+'_test'+str(testn)+'.nc'
    grid_mean_seasonal_ts( plevel, maskfile, betaparamfile, ts_gridmeanfile, datafile, grid_time_i, grid_time_f, window_size_betaparam )


  Interpolated to 14346 coordinates. Nan-estimate: 0.0.  Interpolated to 14346 coordinates. Nan-estimate: 0.0. Gridded estimated 5-day mean saved to outfile.
  Interpolated to 14346 coordinates. Nan-estimate: 0.0. Gridded estimated 5-day mean saved to outfile.
  Interpolated to 14346 coordinates. Nan-estimate: 0.0. Gridded estimated 5-day mean saved to outfile.
  Interpolated to 14346 coordinates. Nan-estimate: 0.0. Gridded estimated 5-day mean saved to outfile.
  Interpolated to 14346 coordinates. Nan-estimate: 0.0. Gridded estimated 5-day mean saved to outfile.
  Interpolated to 14346 coordinates. Nan-estimate: 0.0. Gridded estimated 5-day mean saved to outfile.
  Interpolated to 14346 coordinates. Nan-estimate: 0.0. Gridded estimated 5-day mean saved to outfile.
CPU times: user 12.2 s, sys: 106 ms, total: 12.3 s
Wall time: 14.3 s


In [ ]:
cluster.shutdown()